In [19]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import PIL.ExifTags
import PIL.Image
import json

In [27]:
img1 = cv.imread("./data/IMG_8047.JPG", cv.IMREAD_GRAYSCALE)
img2 = cv.imread("./data/IMG_8048.JPG", cv.IMREAD_GRAYSCALE)

In [18]:

img = PIL.Image.open('data\IMG_8048.JPG')
exif_data = img._getexif()

exif = {
    PIL.ExifTags.TAGS[k]: v
    for k, v in img._getexif().items()
    if k in PIL.ExifTags.TAGS
}

print("by metadata from image exif, f = {} mm, cx = {} pixels, cy = {} pixels".format(exif['FocalLength'], exif['ExifImageHeight'], exif['ExifImageWidth']))


by metadata from image exif, f = 27.182 mm, cx = 3888 pixels, cy = 5184 pixels


In [42]:
sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
pts1 = []
pts2 = []
good_matches = []
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.75*n.distance:
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        good_matches.append([m])
pts1 = np.asarray(pts1)
pts2 = np.asarray(pts2)

In [49]:
camera_intrinsics = np.array([
    [1000, 0, 2500],
    [0, 1000, 2000],
    [0, 0, 1]
])
E, mask = cv.findEssentialMat(pts1, pts2, camera_intrinsics)
_, R, t, mask = cv.recoverPose(E, pts1, pts2, camera_intrinsics)
# obtained R, t, construct P
# [x] x PX = 0
# [x] and P are known
# solve for X
# triangulation
